In [9]:
import jCMIP as jc
import numpy as np
import glob
import os
import copy
import cftime
from netCDF4 import Dataset
import matplotlib.pyplot as plt

Clist = jc.readList('../CMIP6list')

In [10]:
#snames = ['AMV', 'W_S', 'S_S', 'W_M', 'S_M', 'W_N', 'S_N', 'W_S_w', 'S_S_w', 'W_M_w', 'S_M_w', 'W_N_w', 'S_N_w']
#imins  = [-75,   -40,   -60,   -80,   -60,   -60,   -60,   -50,     -60,     -80,      -60,    -60,      -45]
#imaxs  = [0,     -10,   -10,   -30,   -20,   -20,   -20,   -10,     -20,     -30,      -25,    -20,      0]
#jmins  = [0,     10,    20,    25,    35,    45,    50,    10,      15,      25,       30,     40,       45]
#jmaxs  = [60,    35,    35,    50,    50,    70,    70,    25,      30,      35,       45,     60,       60]

#ns = len(snames)

In [11]:
# Details for computation:
EXP          = 'piControl'
var_ts       = 'max_vals_eastward_wind_'
nvar_ts      = 'max_wind'
#var_field    = 'tos_-60to-20E_40to60N'  # Winter North
#var_field    = 'tos_-80to-30E_25to35N'  # Winter Middle
#var_field    = 'tos_-50to-10E_10to25N'  # Winter South
#var_field    = 'tos_-45to0E_45to60N'  # Summer North
#var_field    = 'tos_-60to-25E_30to45N'  # Summer Middle
var_field    = 'tos_-60to-20E_15to30N'  # Summer South
#var_ts       = 'max_lats_eastward_wind_'
#nvar_ts      = 'max_lats'
#var_field    = 'tos_-80to-30E_25to50N'  # Winter Middle
#var_field    = 'tos_-40to-10E_10to35N'  # Winter South
#var_field    = 'tos_-60to-20E_50to70N'  # Summer North
#var_field    = 'tos_-60to-20E_35to50N'  # Summer Middle
#var_field    = 'tos_-60to-10E_20to35N'  # Summer South
var          = 'tos'
season_ts    = 'SON'
lag          = np.arange(-5,10)

# Directory/file details:
dataBen = '/gws/nopw/j04/acsis/bjharvey/storylines/data/JetDiags/'
BenType = '0to60W_fromseasmean'
datadir = '/gws/nopw/j04/acsis/jmecking/CMIP6/'

# Output file:
outfile = (datadir + 'composites/' + EXP + '_' + nvar_ts + '-' + season_ts + '_' + var_field + '_std.nc')

In [12]:
# Find all timeseries files:
Files = glob.glob(datadir + '/*/' + EXP + '/' + var_field + '*.nc' )
print(Files)
nf    = len(Files)
print(nf)

nl = np.size(lag)

# Start counter for seasons (assuming always starts with MAM)
if season_ts == 'MAM':
    sts = 0
elif season_ts == 'JJA':
    sts = 1
elif season_ts == 'SON':
    sts = 2
elif season_ts == 'DJF':
    sts = 3

['/gws/nopw/j04/acsis/jmecking/CMIP6/ACCESS-CM2/piControl/tos_-60to-20E_15to30N_Omon_ACCESS-CM2_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/ACCESS-ESM1-5/piControl/tos_-60to-20E_15to30N_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/BCC-CSM2-MR/piControl/tos_-60to-20E_15to30N_Omon_BCC-CSM2-MR_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/BCC-ESM1/piControl/tos_-60to-20E_15to30N_Omon_BCC-ESM1_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/CAMS-CSM1-0/piControl/tos_-60to-20E_15to30N_Omon_CAMS-CSM1-0_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2/piControl/tos_-60to-20E_15to30N_Omon_CESM2_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-FV2/piControl/tos_-60to-20E_15to30N_Omon_CESM2-FV2_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-WACCM/piControl/tos_-60to-20E_15to30N_Omon_CESM2-WACCM_piControl_r1i1p1f1.nc', '/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-WACCM-FV2

In [13]:
# Create output file if it doesn't exist:

if not os.path.isfile(outfile):
    ncid = Dataset(outfile,'w')
    # Dimensions:
    ncid.createDimension('lag',  nl)
    ncid.createDimension('month',12)
    ncid.createDimension('model',None)

    # Variables:
    ncid.createVariable('lags',    'f8', ('lag',))
    ncid.createVariable('month',   'f8', ('month',))
    ncid.createVariable('models',  'str',('model',))
    ncid.createVariable('mean',    'f8', ('model','month',))
    ncid.createVariable('min_std0','f8', ('model','lag','month',))
    ncid.createVariable('min_std1','f8', ('model','lag','month',))
    ncid.createVariable('max_std0','f8', ('model','lag','month',))
    ncid.createVariable('max_std1','f8', ('model','lag','month',))

    # Data:
    ncid.variables['lags'][:]  = lag
    ncid.variables['month'][:] = np.arange(1,13)
    ncid.close()

    models = []
else:
    # Find out how much has been computed:
    ncid = Dataset(outfile,'r')
    models = ncid.variables['models'][:]
    ncid.close()
    
nm = len(models)

In [14]:
outfile

'/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_wind-SON_tos_-60to-20E_15to30N_std.nc'

In [15]:
infile

'/gws/nopw/j04/acsis/bjharvey/storylines/data/JetDiags/CMIP_MOHC_UKESM1-0-LL_piControl_r1i1p1f2_0to60W_fromseasmean.nc'

In [16]:
# Loop through all files and add if not yet computed:

for ff in range(0,nf):
    fieldfile = Files[ff]
    Model  = Clist[fieldfile.split('_')[-3]]
    ENS    = fieldfile.split('_')[-1].split('.')[0]
    
    mm = (Model.name + '_' + ENS)
    
    if mm in models:
        print('already computed ' + mm)
    else:
        print(fieldfile + ' being computed')
        
        # Read in jet index:
        if Model.name == 'MPI-ESM1-2-HR':
            infile = (dataBen + 'CMIP_' + Model.getInstitute()[2] + '_' +  Model.name + '_' + EXP + '_' + ENS + '_' + BenType + '.nc')
        else:
            infile = (dataBen + 'CMIP_' + Model.getInstitute()[0] + '_' +  Model.name + '_' + EXP + '_' + ENS + '_' + BenType + '.nc')
        ncid = Dataset(infile,'r')
        years_ts = ncid.variables['season_year'][sts::4]  # Assume starts with MAM
        ind_ts   = ncid.variables[var_ts][sts::4,0] # 0 = 850 hPa
        ncid.close()

        nt = np.size(years_ts)
        
        print(nt)
        
        ncid  = Dataset(fieldfile,'r')
        time  = ncid.variables['time'][:]
        cal   = ncid.variables['time'].calendar
        units = ncid.variables['time'].units
        ts    = ncid.variables[var][:]
        ncid.close()
        
        # Divide timeseries into months (assumes first month january and full years):
        ts2   = np.reshape(ts,(int(len(ts)/12),12))
        time2 = np.reshape(time,(int(len(time)/12),12))[:,0]
        ny = len(time2)
        
        # Convert time into years:
        years = copy.deepcopy(time2)
        for tt in range(0,ny):
            years[tt] = cftime.num2date(time2[tt],units,cal).year
        
        # Compute means:
        data_mean = np.mean(ts2,axis=0)
        n_mean = ny

        data_min_0 = np.zeros((nl,12),'float')
        data_max_0 = np.zeros((nl,12),'float')
        indl0 = np.squeeze(np.where(ind_ts < np.mean(ind_ts)))
        indh0 = np.squeeze(np.where(ind_ts > np.mean(ind_ts)))
        n_min0 = np.zeros((nl),'float')
        n_max0 = np.zeros((nl),'float')

        data_min_1 = np.zeros((nl,12),'float')
        data_max_1 = np.zeros((nl,12),'float')
        indl1 = np.squeeze(np.where(ind_ts < np.mean(ind_ts)-np.std(ind_ts)))
        indh1 = np.squeeze(np.where(ind_ts > np.mean(ind_ts)+np.std(ind_ts)))
        n_min1 = np.zeros((nl),'float')
        n_max1 = np.zeros((nl),'float')
        
        for ll in range(0,nl):
            # Low values:
            inds  = indl0 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            for ii in inds:
                ind = np.squeeze(np.where(years_ts[ii] == years))
                if np.size(ind) == 1:
                    data_min_0[ll,:] = data_min_0[ll,:] + ts2[ind,:]
                    n_min0[ll] = n_min0[ll] + 1
            data_min_0[ll,:] = data_min_0[ll,:]/n_min0[ll]

            inds  = indl1 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            for ii in inds:
                ind = np.squeeze(np.where(years_ts[ii] == years))
                if np.size(ind) == 1:
                    data_min_1[ll,:] = data_min_1[ll,:] + ts2[ind,:]
                    n_min1[ll] = n_min1[ll] + 1
            data_min_1[ll,:] = data_min_1[ll,:]/n_min1[ll]

            # High values:
            inds  = indh0 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            for ii in inds:
                ind = np.squeeze(np.where(years_ts[ii] == years))
                if np.size(ind) == 1:
                    data_max_0[ll,:] = data_max_0[ll,:] + ts2[ind,:]
                    n_max0[ll] = n_max0[ll] + 1
            data_max_0[ll,:] = data_max_0[ll,:]/n_max0[ll]

            inds  = indh1 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            for ii in inds:
                ind = np.squeeze(np.where(years_ts[ii] == years))
                if np.size(ind) == 1:
                    data_max_1[ll,:] = data_max_1[ll,:] + ts2[ind,:]
                    n_max1[ll] = n_max1[ll] + 1
            data_max_1[ll,:] = data_max_1[ll,:]/n_max1[ll]
    
        # Write to file:
        ncid = Dataset(outfile,'a')
        ncid.variables['models'  ][nm]       = mm
        ncid.variables['mean'    ][nm,:]   = data_mean
        ncid.variables['min_std0'][nm,:,:] = data_min_0
        ncid.variables['min_std1'][nm,:,:] = data_min_1
        ncid.variables['max_std0'][nm,:,:] = data_max_0
        ncid.variables['max_std1'][nm,:,:] = data_max_1
        ncid.close()
                                   
        nm = nm + 1   

/gws/nopw/j04/acsis/jmecking/CMIP6/ACCESS-CM2/piControl/tos_-60to-20E_15to30N_Omon_ACCESS-CM2_piControl_r1i1p1f1.nc being computed
500
/gws/nopw/j04/acsis/jmecking/CMIP6/ACCESS-ESM1-5/piControl/tos_-60to-20E_15to30N_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1.nc being computed
900
/gws/nopw/j04/acsis/jmecking/CMIP6/BCC-CSM2-MR/piControl/tos_-60to-20E_15to30N_Omon_BCC-CSM2-MR_piControl_r1i1p1f1.nc being computed
600
/gws/nopw/j04/acsis/jmecking/CMIP6/BCC-ESM1/piControl/tos_-60to-20E_15to30N_Omon_BCC-ESM1_piControl_r1i1p1f1.nc being computed
451
/gws/nopw/j04/acsis/jmecking/CMIP6/CAMS-CSM1-0/piControl/tos_-60to-20E_15to30N_Omon_CAMS-CSM1-0_piControl_r1i1p1f1.nc being computed
500
/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2/piControl/tos_-60to-20E_15to30N_Omon_CESM2_piControl_r1i1p1f1.nc being computed
1200
/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-FV2/piControl/tos_-60to-20E_15to30N_Omon_CESM2-FV2_piControl_r1i1p1f1.nc being computed
500
/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-WACCM/piControl/tos

/home/users/jmecking001/miniconda3/envs/working/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in true_divide
/home/users/jmecking001/miniconda3/envs/working/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in true_divide
/home/users/jmecking001/miniconda3/envs/working/lib/python3.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in true_divide
/home/users/jmecking001/miniconda3/envs/working/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide


/gws/nopw/j04/acsis/jmecking/CMIP6/NorESM2-MM/piControl/tos_-60to-20E_15to30N_Omon_NorESM2-MM_piControl_r1i1p1f1.nc being computed
500
/gws/nopw/j04/acsis/jmecking/CMIP6/UKESM1-0-LL/piControl/tos_-60to-20E_15to30N_Omon_UKESM1-0-LL_piControl_r1i1p1f2.nc being computed
1880
